In [41]:
from __future__ import print_function
import pandas as pd
# for plotting
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import numpy as np
pd.pandas.set_option('display.max_columns', None)
#pip install --upgrade gensim
import json
import operator
import collections
import re
import numpy as np
import pandas as pd
import nltk
from gensim.models import word2vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
%matplotlib inline

In [42]:
train_dataset = json.load(open('train.json','r'))
test_dataset = json.load(open('test.json','r'))

In [43]:
ingredients = list()

for recipe in train_dataset:
    for ingredient in recipe[u'ingredients']:
        ingredients.append(ingredient.strip())

cuisines = list()
for recipe in train_dataset:
    cuisines.append(recipe[u'cuisine'].strip())
print("Total recipes in the train corpus",len(cuisines))

Total recipes in the train corpus 39774


# Dictionary of output labels

In [44]:
labels = set(cuisines)
labels = list(labels)
labels = { labels[i] : i for i in range(0, len(labels) ) }
print(labels)

{'british': 0, 'cajun_creole': 1, 'mexican': 2, 'korean': 3, 'thai': 4, 'southern_us': 5, 'indian': 6, 'italian': 7, 'brazilian': 8, 'french': 9, 'japanese': 10, 'moroccan': 11, 'russian': 12, 'irish': 13, 'vietnamese': 14, 'spanish': 15, 'greek': 16, 'chinese': 17, 'jamaican': 18, 'filipino': 19}


In [45]:
print('ingredients in the dataset: ',len((ingredients)))
print('unique ingredients in the dataset: ',len(set(ingredients)))

ingredients in the dataset:  428275
unique ingredients in the dataset:  6714


# Cleaning

In [46]:
import inflect
stemmer = inflect.engine()

recipes_as_sentences_list = list()

for recipe in train_dataset:
    clean_recipe = list()
    for ingredient in recipe['ingredients']:
        ingredient = re.sub(r'[^a-zA-Z\s]+', '', ingredient)
        ingredient =  re.sub(r'\(.*oz.\)|large|medium|small|extra|low-fat|warm|plain|hot|crushed|crumbles|ground|minced|chopped|sliced|heavy|grated|all-purpose|shredded|breakfast|whole|crumbled|self rising|low sodium|flat|diced|unsweetened','', ingredient)
        if stemmer.singular_noun(ingredient) is not False:
            ingredient = stemmer.singular_noun(ingredient)
        words_in_ingredients = nltk.tag.pos_tag(ingredient.split())
        words_in_ingredients=[word[0] for word in words_in_ingredients if word[1] in ('NN')]
        ingredient = " ".join(words_in_ingredients)
        ingredient = ingredient.strip().lower()
        clean_recipe.append(ingredient)
    while("" in clean_recipe) :
        clean_recipe.remove("")
    recipes_as_sentences_list.append(clean_recipe)

In [47]:
u_ingredients = set()
for recipe in recipes_as_sentences_list:
    for ingredient in recipe:
        u_ingredients.add(ingredient)
print('unique ingredients after cleaning the dataset: ',len(u_ingredients))

unique ingredients after cleaning the dataset:  3903


In [48]:
ingredients_removed_in_cleaning = u_ingredients.difference(set(ingredients))
print(len(ingredients_removed_in_cleaning))

1719


In [49]:
num_features = 300              
min_word_count = 1                   
num_workers = 4
context = 10                                                                                

model = word2vec.Word2Vec(recipes_as_sentences_list, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)

In [50]:
print("Before cleaning: ")
print(train_dataset[12342]["ingredients"])
print("After cleaning: ")
print(recipes_as_sentences_list[12342])
print("=======================================================")
print("Before cleaning: ")
print(train_dataset[122]["ingredients"])
print("After cleaning: ")
print(recipes_as_sentences_list[122])

Before cleaning: 
['light soy sauce', 'peeled shrimp', 'brown sugar', 'green onions', 'chicken stock', 'fresh ginger root', 'boneless pork loin', 'chinese rice wine', 'wonton wrappers']
After cleaning: 
['soy sauce', 'shrimp', 'brown sugar', 'onion', 'chicken stock', 'ginger root', 'boneless pork loin', 'rice wine', 'wonton wrapper']
Before cleaning: 
['tomato sauce', 'ground black pepper', 'garlic', 'scallions', 'chipotles in adobo', 'avocado', 'dried thyme', 'instant white rice', 'cilantro leaves', 'coconut milk', 'water', 'red beans', 'chopped celery', 'skinless chicken thighs', 'onions', 'lime zest', 'lime juice', 'lime wedges', 'salt', 'carrots']
After cleaning: 
['tomato sauce', 'pepper', 'garlic', 'scallion', 'chipotle adobo', 'avocado', 'thyme', 'rice', 'cilantro leaf', 'coconut milk', 'water', 'bean', 'celery', 'skinless chicken thigh', 'onion', 'lime zest', 'lime juice', 'lime wedge', 'salt', 'carrot']


In [51]:
model.similarity('ginger root', 'ginger')

0.9395316

In [52]:
print(model.wv.vectors.shape)
vocab = model.wv.vocab
#print(model['olive'])

(3903, 300)


In [53]:
ingredients_new = list()
for recipe in recipes_as_sentences_list:
    for ingredient in recipe:
        ingredients_new.append(ingredient.strip())
print(len((ingredients_new)))
ingredients_new_set = set(ingredients_new)
print(len((ingredients_new_set)))

#so there are no unknown 

416965
3903


In [54]:
unknown_ingredients = []
for ingredient in ingredients_new_set:
    if ingredient not in vocab.keys():
        unknown_ingredients.append(ingredient)

In [55]:
print(len(unknown_ingredients))

0


In [56]:
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    sent = [x for x in sent if x not in unknown_ingredients]
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model[w]
            else:
                sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
    return np.asarray(sent_vec) / numw

In [57]:
recipes_as_context_vectors_of_ingredients=[]
for sentence in recipes_as_sentences_list:
    recipes_as_context_vectors_of_ingredients.append(sent_vectorizer(sentence, model))

In [58]:
print("Length of vector", len(recipes_as_context_vectors_of_ingredients[0]))
print(recipes_as_sentences_list[0])
print("Context Vectors taindataset count records: ", len(recipes_as_context_vectors_of_ingredients))
print("Original taindataset count records: ",len(recipes_as_sentences_list))

Length of vector 300
['romaine lettuce', 'olive', 'grape tomato', 'garlic', 'pepper', 'onion', 'feta cheese']
Context Vectors taindataset count records:  39774
Original taindataset count records:  39774


# Update json tarindataset

In [59]:
for i,recipe in enumerate(train_dataset):
    recipe[u'ingredients'] = recipes_as_context_vectors_of_ingredients[i]
    recipe[u'cuisine'] = labels[recipe[u'cuisine']]

# Convert Json to Pandas dataframe: Only for understanding and taken output variable from this dataframe

In [60]:
from pandas.io.json import json_normalize
train_dataset_new = json_normalize(train_dataset)
train_dataset_new.drop(['id'],axis=1,inplace=True)
train_dataset_new.head()

,cuisine,ingredients
0,16,"[0.28586197, -0.09290139, -0.10495607, -0.1410..."
1,5,"[0.20361829, 0.092594296, -0.11171341, -0.1515..."
2,19,"[0.08699065, 0.051300272, 0.01406965, -0.26668..."
3,6,"[0.56616384, 0.20685188, -0.044593178, -0.2793..."
4,6,"[0.10824637, 0.15037799, 0.0628284, -0.1791812..."


# Machine Learning

In [61]:
data = pd.DataFrame(recipes_as_context_vectors_of_ingredients)
print(data.shape)
data.head()

(39774, 300)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.285862,-0.092901,-0.104956,-0.141079,-0.343134,0.018658,-0.329705,-0.253953,-0.107836,0.275170,0.126640,-0.164504,0.242465,-0.156059,0.069863,-0.371540,-0.186931,-0.457091,-0.283191,0.018625,0.216196,-0.047563,0.054361,0.331155,-0.221449,0.118537,0.309211,0.099856,-0.195914,-0.216138,0.148422,0.111567,-0.243724,-0.301540,0.153315,0.226266,-0.168114,0.038124,-0.377165,-0.104259,-0.245899,-0.034031,0.186209,-0.246264,-0.355434,-0.021470,-0.128582,0.245688,0.456542,0.075975,-0.102264,0.066549,-0.158218,-0.314325,0.225521,-0.224753,-0.022723,-0.083077,0.166481,0.009899,-0.473658,-0.497112,0.032715,-0.091476,-0.164462,0.171175,0.410027,0.197244,0.495246,-0.095109,0.531430,0.021766,-0.332679,-0.048326,0.279605,-0.022727,0.165361,-0.234565,0.276868,0.110088,0.091306,-0.506916,0.450488,-0.025608,-0.195939,0.263726,0.198808,0.190501,0.238037,0.161614,-0.335157,0.261250,0.366735,-0.320798,0.100499,0.323603,0.206073,0.142174,-0.167733,0.040372,-0.107068,-0.101688,-0.283618,-0.490180,-0.082247,-0.248786,0.073676,-0.392439,0.318291,0.009560,0.045009,0.077192,0.315131,0.101433,0.033035,-0.214375,-0.020706,0.170736,0.102457,-0.132621,0.036060,-0.334531,0.381246,-0.254584,0.082054,-0.383621,-0.089251,0.037412,-0.096276,0.061637,0.219466,-0.288311,0.308858,-0.382105,-0.525282,0.100674,0.017538,0.204352,0.429220,0.464965,0.477380,0.217622,0.147840,0.488478,0.073497,-0.128959,0.139289,0.106418,0.288603,-0.511372,0.066647,-0.127446,-0.046222,0.210256,-0.704312,0.012463,0.436102,0.041509,-0.137014,-0.288830,-0.290843,0.069458,-0.056271,0.170431,0.200217,-0.207172,-0.235331,0.080113,-0.128064,-0.309254,0.199523,0.306442,0.345948,-0.325753,-0.160482,0.480860,-0.206121,-0.029315,-0.065020,0.219120,0.541726,-0.149281,-0.086914,-0.045517,0.417162,0.194302,0.416090,-0.087867,-0.105092,-0.209816,0.194123,0.078157,-0.419996,-0.064581,0.298767,-0.012142,-0.230738,0.116464,0.548734,0.039930,-0.050884,-0.299697,0.354323,0.053719,-0.248611,-0.156978,0.144317,-0.033441,-0.133375,0.318379,0.103274,-0.032953,-0.013313,0.061050,-0.053579,0.615861,-0.205658,0.090130,0.210690,-0.151209,0.239364,-0.315420,-0.266888,0.114705,-0.001046,-0.127042,-0.326912,-0.656359,0.327963,0.084491,-0.351521,0.397267,0.222389,0.096462,0.065139,-0.065133,-0.180603,0.362301,0.122646,-0.202959,0.139354,-0.070015,-0.463280,-0.041579,-0.261727,-0.469859,-0.256715,0.307068,-0.200408,-0.152571,0.095478,-0.561029,-0.068957,0.167482,-0.132480,-0.199525,-0.009716,-0.063114,-0.444418,0.087899,-0.248784,-0.401089,-0.162596,-0.252509,0.207560,-0.360343,0.078712,-0.434182,-0.072858,0.292766,0.145144,0.791459,-0.378336,-0.102891,0.398605,-0.039855,-0.157138,-0.376547,0.213217,0.234560,0.068015,-0.245633,-0.344618,-0.066779,0.127250,0.090009,0.080086,-0.280653,0.233545,0.371713,0.194261,-0.013065,0.269459,-0.091065,-0.206738,-0.206610,-0.142062,-0.137695,0.060239,-0.257914
1,0.203618,0.092594,-0.111713,-0.151551,-0.092854,0.1

In [62]:
label_data = train_dataset_new.cuisine
print(label_data.shape)
label_data.head()
final_data = pd.concat([data, label_data], axis=1)
print(final_data.shape)
final_data.head()

(39774,)
(39774, 301)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,cuisine
0,0.285862,-0.092901,-0.104956,-0.141079,-0.343134,0.018658,-0.329705,-0.253953,-0.107836,0.275170,0.126640,-0.164504,0.242465,-0.156059,0.069863,-0.371540,-0.186931,-0.457091,-0.283191,0.018625,0.216196,-0.047563,0.054361,0.331155,-0.221449,0.118537,0.309211,0.099856,-0.195914,-0.216138,0.148422,0.111567,-0.243724,-0.301540,0.153315,0.226266,-0.168114,0.038124,-0.377165,-0.104259,-0.245899,-0.034031,0.186209,-0.246264,-0.355434,-0.021470,-0.128582,0.245688,0.456542,0.075975,-0.102264,0.066549,-0.158218,-0.314325,0.225521,-0.224753,-0.022723,-0.083077,0.166481,0.009899,-0.473658,-0.497112,0.032715,-0.091476,-0.164462,0.171175,0.410027,0.197244,0.495246,-0.095109,0.531430,0.021766,-0.332679,-0.048326,0.279605,-0.022727,0.165361,-0.234565,0.276868,0.110088,0.091306,-0.506916,0.450488,-0.025608,-0.195939,0.263726,0.198808,0.190501,0.238037,0.161614,-0.335157,0.261250,0.366735,-0.320798,0.100499,0.323603,0.206073,0.142174,-0.167733,0.040372,-0.107068,-0.101688,-0.283618,-0.490180,-0.082247,-0.248786,0.073676,-0.392439,0.318291,0.009560,0.045009,0.077192,0.315131,0.101433,0.033035,-0.214375,-0.020706,0.170736,0.102457,-0.132621,0.036060,-0.334531,0.381246,-0.254584,0.082054,-0.383621,-0.089251,0.037412,-0.096276,0.061637,0.219466,-0.288311,0.308858,-0.382105,-0.525282,0.100674,0.017538,0.204352,0.429220,0.464965,0.477380,0.217622,0.147840,0.488478,0.073497,-0.128959,0.139289,0.106418,0.288603,-0.511372,0.066647,-0.127446,-0.046222,0.210256,-0.704312,0.012463,0.436102,0.041509,-0.137014,-0.288830,-0.290843,0.069458,-0.056271,0.170431,0.200217,-0.207172,-0.235331,0.080113,-0.128064,-0.309254,0.199523,0.306442,0.345948,-0.325753,-0.160482,0.480860,-0.206121,-0.029315,-0.065020,0.219120,0.541726,-0.149281,-0.086914,-0.045517,0.417162,0.194302,0.416090,-0.087867,-0.105092,-0.209816,0.194123,0.078157,-0.419996,-0.064581,0.298767,-0.012142,-0.230738,0.116464,0.548734,0.039930,-0.050884,-0.299697,0.354323,0.053719,-0.248611,-0.156978,0.144317,-0.033441,-0.133375,0.318379,0.103274,-0.032953,-0.013313,0.061050,-0.053579,0.615861,-0.205658,0.090130,0.210690,-0.151209,0.239364,-0.315420,-0.266888,0.114705,-0.001046,-0.127042,-0.326912,-0.656359,0.327963,0.084491,-0.351521,0.397267,0.222389,0.096462,0.065139,-0.065133,-0.180603,0.362301,0.122646,-0.202959,0.139354,-0.070015,-0.463280,-0.041579,-0.261727,-0.469859,-0.256715,0.307068,-0.200408,-0.152571,0.095478,-0.561029,-0.068957,0.167482,-0.132480,-0.199525,-0.009716,-0.063114,-0.444418,0.087899,-0.248784,-0.401089,-0.162596,-0.252509,0.207560,-0.360343,0.078712,-0.434182,-0.072858,0.292766,0.145144,0.791459,-0.378336,-0.102891,0.398605,-0.039855,-0.157138,-0.376547,0.213217,0.234560,0.068015,-0.245633,-0.344618,-0.066779,0.127250,0.090009,0.080086,-0.280653,0.233545,0.371713,0.194261,-0.013065,0.269459,-0.091065,-0.206738,-0.206610,-0.142062,-0.137695,0.060239,-0.257914,16
1,0.203618,0.092594,-0.111713,-0.151551,-0

In [63]:
print("Indices of NaN: ", pd.isnull(data).any(1).nonzero()[0])
final_data= final_data.dropna()
print("Indices of NaN: ", pd.isnull(final_data).any(1).nonzero()[0])
output_label = final_data['cuisine']
final_data.drop(['cuisine'],axis=1,inplace=True)

Indices of NaN:  [13178 23512]
Indices of NaN:  []


In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_data, output_label, test_size=0.15, random_state=42)

In [65]:
#np.where(np.isnan(data))
#np.isnan(data)
#print("Indices of NaN: ", pd.isnull(data).any(1).nonzero()[0])
#print(data.loc[[13178]])
#print(data.loc[[23512]])
#data.drop(data.index[13178],inplace=True)
#data.drop(data.index[23512],inplace=True)
#label_data.drop(data.index[13178],inplace=True)
#label_data.drop(data.index[23512],inplace=True)
#label_data.drop(data.index[23511],inplace=True)
#print("Indices of NaN: ", pd.isnull(data).any(1).nonzero()[0])

In [67]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

print("train: ",clf.score(X_train,y_train))
print("test: ",clf.score(X_test,y_test))

train:  0.6521031769508371
test:  0.6345960442507542


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(X_train, y_train)

In [ ]:
print('Best score for training data:', svm_model.best_score_,"\n") 